In [1]:
import cv2
import numpy as np
import scipy
from scipy.misc import imread
import pickle
import random
import os
import matplotlib.pyplot as plt

In [10]:
image_path = 'queryImages/sample_1.jpg'
database_path = 'referenceImages/'

In [11]:
import glob
import cv2

# Feature extractor
def extract_features(image_path, vector_size=32):
    image = imread(image_path, mode="RGB")
    try:
        alg = cv2.KAZE_create()
        # Finding image keypoints
        # use glob to grab the image paths and loop over them
        imagePaths = [f for f in glob.glob('directory/*.png')]
        kps = alg.detect(image)
        # Getting first 32 of them. 
        # Number of keypoints is varies depend on image size and color pallet
        # Sorting them based on keypoint response value(bigger is better)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # computing descriptors vector
        kps, dsc = alg.compute(image, kps)
        # Flatten all of them in one big vector - our feature vector
        dsc = dsc.flatten()
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if dsc.size < needed_size:
            # if we have less the 32 descriptors then just adding zeros at the
            # end of our feature vector
             dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    except cv2.error as e:
        print ('Error: ', e)
        return None

    return dsc


def batch_extractor(database_path, pickled_db_path="kaze_features.pck"):
    files = [os.path.join(database_path, p) for p in sorted(os.listdir(database_path))]

    result = {}
    for f in files:
        print ('Extracting features from image %s' % f)
        name = f.split('/')[-1].lower()
        result[name] = extract_features(f)
    
    # saving all our feature vectors in pickled file
    with open(pickled_db_path, 'wb') as fp:
        pickle.dump(result, fp)

In [12]:
class Matcher(object):

    def __init__(self, pickled_db_path="kaze_features.pck"):
        with open(pickled_db_path, 'rb') as fp:
            self.data = pickle.load(fp)
            self.names = []
            self.matrix = []
        for k, v in self.data.items():
            self.names.append(k)
            self.matrix.append(v)
        self.matrix = np.array(self.matrix)
        self.names = np.array(self.names)

    def cos_cdist(self, vector):
        # getting cosine distance between search image and images database
        v = vector.reshape(1, -1)
        return scipy.spatial.distance.cdist(self.matrix, v, 'cosine').reshape(-1)

    def match(self, image_path, topn=5):
        features = extract_features(image_path)
        img_distances = self.cos_cdist(features)
        # getting top 5 records
        nearest_ids = np.argsort(img_distances)[:topn].tolist()
        nearest_img_paths = self.names[nearest_ids].tolist()

        return nearest_img_paths, img_distances[nearest_ids].tolist()

In [ ]:

def show_img(path):
    img = imread(path, mode="RGB")
    plt.imshow(img)
    plt.show()
    
def run():
    images_path = database_path
    files = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]
    # getting 3 random images 
    sample = random.sample(files, 3)
    
    batch_extractor(images_path)

    ma = Matcher('kaze_features.pck')
    
    for s in sample:
        print ('Query image ==========================================')
        show_img(s)
        names, match = ma.match(s, topn=3)
        print ('Result images ========================================')
        for i in range(3):
            # we got cosine distance, less cosine distance between vectors
            # more they similar, thus we subtruct it from 1 to get match value
            print ('Match %s' % (1-match[i]))
            show_img(os.path.join(images_path, names[i]))

run()

Extracting features from image referenceImages/1792-bottle-in-bond-[size].png


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Extracting features from image referenceImages/1792-full-proof-bourbon-[size].png
Extracting features from image referenceImages/1792-high-rye-[size].png
Extracting features from image referenceImages/1792-single-barrel-bourbon-[size].png
Extracting features from image referenceImages/1792-small-batch-bourbon-[size].png
Extracting features from image referenceImages/1800-anejo-[size].png
Extracting features from image referenceImages/1800-reposado-[size].png
Extracting features from image referenceImages/1800-silver-[size].png
Extracting features from image referenceImages/1921-anejo-[size].png
Extracting features from image referenceImages/1921-reposado-[size].png
Extracting features from image referenceImages/Ad-Laws-bottled-in-bond.png
Extracting features from image referenceImages/Ad-Laws-straight-bourbon-whiskey.png
Extracting features from image referenceImages/Angel's Kentucky Straight Bourbon.png
Extracting features from image referenceImages/Angels-Envy-Cask-Strength.png
Extra

Extracting features from image referenceImages/MYERSS_RUM_ORIGINAL_DARK_RUM_80PF_750ML.png
Extracting features from image referenceImages/Mezcales de Leyenda - durango.png
Extracting features from image referenceImages/Mezcales de Leyenda - guerrero.png
Extracting features from image referenceImages/Mezcales de Leyenda - oaxaca.png
Extracting features from image referenceImages/Milagro Anejo Select Barrel.png
Extracting features from image referenceImages/Milagro Reposado.png
Extracting features from image referenceImages/Milagro Silver.png
Extracting features from image referenceImages/Mount Gay XO Reserve Cask.png
Extracting features from image referenceImages/NIKKA-COFFEY-GRAIN-MALT-70CL.png
Extracting features from image referenceImages/Nikka-Coffey-Malt.png
Extracting features from image referenceImages/Nikka-from-the-barrel-whisky-500ml.png
Extracting features from image referenceImages/Oban14-750ml.png
Extracting features from image referenceImages/Oban_Distiller_s_Edition_750ml